# 🧠 Project 01 — Artificial Intelligence 2025/26  
## Class Timetable (CSP Problem)

---

### 1. Introduction
> This project aims to develop an intelligent agent capable of generating class timetables for the undergraduate courses at the Polytechnic Institute of Cávado and Ave (IPCA).  
> Creating timetables is a complex and time-consuming task that involves multiple constraints related to professors, subjects, classrooms, and student groups. Traditionally, this process requires significant time and effort from the administrative team.  
> In this context, the project intends to apply Artificial Intelligence techniques, specifically Constraint Satisfaction Problems (CSP), to automate the creation of viable and optimized timetables, respecting the imposed conditions and minimizing conflicts.  
> The project will be implemented in Python, using the `python-constraint` library, and fully documented in a Jupyter Notebook, as per the guidelines of the Artificial Intelligence (AI) course.

**Group members:**
- Pedro Ribeiro — student number 27960  
- Ricardo Fernandes — student number 27961  
- Carolina Branco — student number 27983  
- João Barbosa — student number 27964  
- Diogo Abreu — student number 27975  

---

### 2. Goal Formulation

The main goal of this project is to **design an intelligent agent** capable of automatically generating valid class timetables for undergraduate courses at IPCA.  

The agent must assign **courses to time slots and rooms**, ensuring that no scheduling conflicts occur and that all academic and logistical constraints are satisfied.  

#### Limitations and Constraints
The timetable generation process is subject to several limitations, including:

- **Teacher availability** — each lecturer may have unavailable time slots.  
- **Room capacity and restrictions** — some courses must occur in specific rooms (e.g., labs).  
- **Course assignment** — each class is associated with a specific set of courses.  
- **Online lessons** — some sessions may occur online and not require a physical room.  
- **Non-overlapping sessions** — a class, teacher, or room cannot be used in two sessions simultaneously.  
- **Class duration** — each lesson lasts 2 hours.  
- **Weekly lessons per class** — all classes have 10 lessons per week.  
- **Lessons per course** — each course may have 1 or 2 lessons per week.  
- **Daily lesson limit** — a class cannot have more than 3 lessons per day.  

#### Expected Results
The system should produce a **valid timetable solution** that:
- Assigns every course session to a valid time slot and room.  
- Respects all **hard constraints**.  
- Minimizes conflicts and overlaps between classes and teachers.  
- Optionally considers **soft constraints** to optimize the timetable.

---

### 3. Problem Formulation (CSP)

The timetable problem was modeled as a **Constraint Satisfaction Problem (CSP)** using the `python-constraint` library.

#### Variables
Each **lesson** of a course is represented by two variables:
- `<course>_<lesson>_slot` — the time slot assigned to that lesson.  
- `<course>_<lesson>_room` — the classroom assigned to that lesson.

#### Domains
- **Time slots:** `1–20` (representing possible periods in the week).  
- **Rooms:** `{Lab01, Room1, Room2}`, unless specific restrictions apply.  

#### Constraints

**Hard Constraints (mandatory):**
- Classes last 2 hours.  
- All classes have 10 lessons per week.  
- Each course may have 1 or 2 lessons per week.  
- A class cannot have more than 3 lessons per day.  
- Teacher availability — a course cannot be scheduled in time slots where its lecturer is unavailable.  
- Room restrictions — some courses are restricted to specific rooms (e.g., labs).  
- No overlapping classes — no class, teacher, or room can appear in two lessons at the same time.  

**Soft Constraints (preferred, can be violated if necessary):**
- Lessons of the same course must be scheduled on distinct days.  
- Each class should have only four days of lessons per week.  
- Lessons within the same day should be consecutive. 

#### Additional Constraints (optional improvements)

**Hard Constraints (additional):**
- When online classes are scheduled, limited to a maximum of three per course, they must be scheduled on the same day.  
- Some classes are required to be assigned to a specific classroom (already partly implemented via `roomrestrictions`).  

**Soft Constraints (additional):**
- The number of classrooms used by each class should be minimized.  

#### Heuristics
The CSP solver may apply **variable ordering heuristics** (e.g., most constrained variable first) and **domain reduction** to improve efficiency.  
Further heuristics (e.g., least constraining value) may be explored in later iterations.

---

### 5. Repository Link
> 🔗 **GitHub Repository:** [https://github.com/diogooaabreu/IA25_P01_G4.git](https://github.com/diogooaabreu/IA25_P01_G4.git)

---

### 4. Implementation

> The implementation is based on the `python-constraint` library.  
> The dataset is read from `datasets/timetable_dataset.txt`, and constraints are dynamically built from the data.  
> Below is the initial implementation of the CSP model:

---




In [ ]:
import json
import time
from collections import Counter
from ortools.sat.python import cp_model


# ======================================================
# 1. BASIC CONFIGURATION AND CONSTANTS
# ======================================================

rooms = ["Lab01", "Room1", "Room2", "Online"]
lessons_per_course = 2
slots = list(range(1, 21))

room_to_int = {r: i for i, r in enumerate(rooms)}
int_to_room = {i: r for r, i in room_to_int.items()}


# ======================================================
# 2. HELPER FUNCTIONS
# ======================================================

def build_teacher_mappings(courses_to_lecturers):
    """Create integer mappings for teacher identifiers.

    Args:
        courses_to_lecturers (dict): Mapping of teachers to their respective courses.

    Returns:
        tuple: A pair of dictionaries (teacher_to_int, int_to_teacher)
               for mapping between teacher names and integer indices.
    """
    teachers = sorted(courses_to_lecturers.keys())
    t2i = {t: i for i, t in enumerate(teachers)}
    i2t = {i: t for t, i in t2i.items()}
    return t2i, i2t


def get_day_from_slot(slot):
    """Return the day index corresponding to a slot number.

    Slots are numbered sequentially, with 4 slots per day.
    Monday = 0, Tuesday = 1, ..., Friday = 4.

    Args:
        slot (int): The slot number.

    Returns:
        int: The day index (0–4).
    """
    return (slot - 1) // 4


# ======================================================
# 3. DATA LOADING
# ======================================================

dataset_path = "datasets/timetable_dataset.txt"

courses_assigned_to_classes = {}
courses_assigned_to_lecturers = {}
timeslot_restrictions = {}
roomrestrictions = {}
online_classes = {}
all_courses_with_variables = set()

try:
    with open(dataset_path, "r", encoding="utf-8-sig") as f:
        lines = f.readlines()

    reading_section = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        elif line.startswith("#cc"):
            reading_section = "cc"
        elif line.startswith("#dsd"):
            reading_section = "dsd"
        elif line.startswith("#tr"):
            reading_section = "tr"
        elif line.startswith("#rr"):
            reading_section = "rr"
        elif line.startswith("#oc"):
            reading_section = "oc"
        elif line.startswith("#"):
            reading_section = None
        else:
            if reading_section == "cc":
                parts = line.split()
                courses_assigned_to_classes[parts[0]] = parts[1:]
                for course in parts[1:]:
                    all_courses_with_variables.add(course)
            elif reading_section == "dsd":
                parts = line.split()
                courses_assigned_to_lecturers[parts[0]] = parts[1:]
            elif reading_section == "tr":
                parts = line.split()
                timeslot_restrictions[parts[0]] = list(map(int, parts[1:]))
            elif reading_section == "rr":
                parts = line.split()
                roomrestrictions[parts[0]] = parts[1:]
            elif reading_section == "oc":
                parts = line.split()
                online_classes[parts[0]] = list(map(int, parts[1:]))

except FileNotFoundError:
    """Simulated dataset used when the file is not found."""
    print(f"ERROR: File not found at path: {dataset_path}")
    print("Using simulated data to continue the example.")
    courses_assigned_to_classes = {
        't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15',
                'UC21', 'UC22', 'UC23', 'UC24', 'UC25',
                'UC31', 'UC32', 'UC33', 'UC34', 'UC35']
    }
    courses_assigned_to_lecturers = {
        'jo': ['UC11', 'UC21', 'UC22', 'UC31'],
        'mike': ['UC12', 'UC23', 'UC32'],
        'rob': ['UC13', 'UC14', 'UC24', 'UC33'],
        'sue': ['UC15', 'UC25', 'UC34', 'UC35']
    }
    timeslot_restrictions = {
        'mike': [13, 14, 15, 16, 17, 18, 19, 20],
        'rob': [1, 2, 3, 4],
        'sue': [9, 10, 11, 12, 17, 18, 19, 20]
    }
    roomrestrictions = {'UC14': ['Lab01'], 'UC22': ['Lab01']}
    online_classes = {'UC21': [2], 'UC31': [2]}
    all_courses_with_variables = set(courses_assigned_to_classes['t01'])


# ======================================================
# 4. MODEL CREATION
# ======================================================

model = cp_model.CpModel()
teacher_to_int, int_to_teacher = build_teacher_mappings(courses_assigned_to_lecturers)

slot_vars = {}
room_vars = {}
teacher_vars = {}
day_vars = {}


# ======================================================
# 5. VARIABLE DEFINITIONS
# ======================================================

for class_name, courses in courses_assigned_to_classes.items():
    for course in courses:
        for lesson in range(1, lessons_per_course + 1):
            key = (course, lesson)
            s = model.NewIntVar(1, 20, f"{course}_{lesson}_slot")
            slot_vars[key] = s

            if course in roomrestrictions:
                allowed_rooms = roomrestrictions[course]
                allowed_room_idxs = [room_to_int[r] for r in allowed_rooms]
                rvar = model.NewIntVar(min(allowed_room_idxs),
                                       max(allowed_room_idxs),
                                       f"{course}_{lesson}_room_tmp")
                r = model.NewIntVar(min(room_to_int.values()),
                                    max(room_to_int.values()),
                                    f"{course}_{lesson}_room")
                model.AddAllowedAssignments([r], [[v] for v in allowed_room_idxs])
            else:
                r = model.NewIntVar(min(room_to_int.values()),
                                    max(room_to_int.values()),
                                    f"{course}_{lesson}_room")
            room_vars[key] = r

            teachers_for_course = [
                t for t, c_list in courses_assigned_to_lecturers.items() if course in c_list
            ]
            if teachers_for_course:
                teacher_idxs = [teacher_to_int[t] for t in teachers_for_course]
                tvar = model.NewIntVar(min(teacher_to_int.values()),
                                       max(teacher_to_int.values()),
                                       f"{course}_{lesson}_teacher")
                model.AddAllowedAssignments([tvar], [[v] for v in teacher_idxs])
                teacher_vars[key] = tvar

            dvar = model.NewIntVar(0, 4, f"{course}_{lesson}_day")
            day_vars[key] = dvar


# ======================================================
# 6. CONSTRAINTS
# ======================================================

slot_day_allowed = [[s, get_day_from_slot(s)] for s in slots]
for key, s_var in slot_vars.items():
    d_var = day_vars[key]
    model.AddAllowedAssignments([s_var, d_var], slot_day_allowed)

# Prevent consecutive lessons of the same course
for class_name, courses in courses_assigned_to_classes.items():
    for course in courses:
        lesson_keys = [(course, lesson) for lesson in range(1, lessons_per_course + 1)]
        for i in range(len(lesson_keys)):
            for j in range(i + 1, len(lesson_keys)):
                k1, k2 = lesson_keys[i], lesson_keys[j]
                s1, s2 = slot_vars[k1], slot_vars[k2]
                model.Add(s1 - s2 != 1)
                model.Add(s2 - s1 != 1)

# Prevent room conflicts and consecutive usage
all_course_list = list(all_courses_with_variables)
for i in range(len(all_course_list)):
    for j in range(i + 1, len(all_course_list)):
        c1, c2 = all_course_list[i], all_course_list[j]
        for l1 in range(1, lessons_per_course + 1):
            for l2 in range(1, lessons_per_course + 1):
                key1, key2 = (c1, l1), (c2, l2)
                if key1 in room_vars and key2 in room_vars:
                    r1, r2 = room_vars[key1], room_vars[key2]
                    s1, s2 = slot_vars[key1], slot_vars[key2]
                    same_room = model.NewBoolVar(f"same_room_{c1}_{l1}__{c2}_{l2}")
                    model.Add(r1 == r2).OnlyEnforceIf(same_room)
                    model.Add(r1 != r2).OnlyEnforceIf(same_room.Not())
                    model.Add(s1 - s2 != 1).OnlyEnforceIf(same_room)
                    model.Add(s2 - s1 != 1).OnlyEnforceIf(same_room)

# Apply teacher availability constraints
for teacher, courses in courses_assigned_to_lecturers.items():
    unavailable_slots = timeslot_restrictions.get(teacher, [])
    for course in courses:
        if course in all_courses_with_variables:
            for lesson in range(1, lessons_per_course + 1):
                key = (course, lesson)
                if key in slot_vars and key in teacher_vars:
                    tvar = teacher_vars[key]
                    svar = slot_vars[key]
                    for unavailable in unavailable_slots:
                        if teacher in teacher_to_int:
                            t_idx = teacher_to_int[teacher]
                            is_this_teacher = model.NewBoolVar(f"is_{teacher}_{course}_{lesson}")
                            model.Add(tvar == t_idx).OnlyEnforceIf(is_this_teacher)
                            model.Add(tvar != t_idx).OnlyEnforceIf(is_this_teacher.Not())
                            model.Add(svar != unavailable).OnlyEnforceIf(is_this_teacher)

# Ensure unique time slots per class
for class_name, courses in courses_assigned_to_classes.items():
    lesson_slots = [slot_vars[(course, lesson)]
                    for course in courses
                    for lesson in range(1, lessons_per_course + 1)
                    if (course, lesson) in slot_vars]
    if lesson_slots:
        model.AddAllDifferent(lesson_slots)

# Ensure unique time slots per teacher
for teacher, courses in courses_assigned_to_lecturers.items():
    lesson_slots = [slot_vars[(course, lesson)]
                    for course in courses
                    for lesson in range(1, lessons_per_course + 1)
                    if (course, lesson) in slot_vars and (course, lesson) in teacher_vars]
    if lesson_slots:
        model.AddAllDifferent(lesson_slots)

# Force online courses to be in the "Online" room
for course, week_indexes in online_classes.items():
    if course in all_courses_with_variables:
        for lesson in week_indexes:
            key = (course, lesson)
            if key in room_vars:
                model.Add(room_vars[key] == room_to_int["Online"])

# Limit number of lessons per day for each class
for class_name, courses in courses_assigned_to_classes.items():
    lesson_day_vars = [day_vars[(course, lesson)]
                       for course in courses
                       for lesson in range(1, lessons_per_course + 1)
                       if (course, lesson) in day_vars]
    if lesson_day_vars:
        for d in range(5):
            indicators = []
            for idx, dvar in enumerate(lesson_day_vars):
                b = model.NewBoolVar(f"{class_name}_day{d}_lesson{idx}")
                model.Add(dvar == d).OnlyEnforceIf(b)
                model.Add(dvar != d).OnlyEnforceIf(b.Not())
                indicators.append(b)
            model.Add(sum(indicators) <= 3)


# ======================================================
# 7. DECISION STRATEGIES (HEURISTICS)
# ======================================================

slot_var_list = list(slot_vars.values())
if slot_var_list:
    model.AddDecisionStrategy(
        slot_var_list,
        cp_model.CHOOSE_MIN_DOMAIN_SIZE,
        cp_model.SELECT_MIN_VALUE
    )

from src.helpers import escolher_heuristica_variaveis, escolher_heuristica_valores

heuristica_vars_nome = "mrv"
heuristica_vals_nome = "lcv"
MAX_TIME_SECONDS = 30.0

var_strategy = escolher_heuristica_variaveis(heuristica_vars_nome)
val_strategy = escolher_heuristica_valores(heuristica_vals_nome)

model.AddDecisionStrategy(
    [*slot_vars.values(), *room_vars.values(), *teacher_vars.values()],
    var_strategy,
    val_strategy
)


# ======================================================
# 8. SOLVER CONFIGURATION AND EXECUTION
# ======================================================

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = MAX_TIME_SECONDS
solver.parameters.enumerate_all_solutions = True


# ======================================================
# 9. SOLUTION COLLECTOR CLASS
# ======================================================

class SolutionCollector(cp_model.CpSolverSolutionCallback):
    """Custom solution callback for collecting and evaluating solutions.

    The callback stores all found solutions, computes a heuristic
    score based on room changes for each teacher, and retains
    the best (lowest-score) solution.
    """

    def __init__(self, slot_vars, room_vars, teacher_vars):
        """Initialize the solution collector.

        Args:
            slot_vars (dict): Mapping of (course, lesson) to slot variables.
            room_vars (dict): Mapping of (course, lesson) to room variables.
            teacher_vars (dict): Mapping of (course, lesson) to teacher variables.
        """
        super().__init__()
        self.slot_vars = slot_vars
        self.room_vars = room_vars
        self.teacher_vars = teacher_vars
        self.solution_count = 0
        self.best_solution = None
        self.best_score = None

    def OnSolutionCallback(self):
        """Handle each found solution and keep the best one based on score."""
        self.solution_count += 1
        slots = {k: self.Value(v) for k, v in self.slot_vars.items()}
        rooms = {k: self.Value(v) for k, v in self.room_vars.items()}
        teachers = {k: self.Value(v) for k, v in self.teacher_vars.items()}
        score = 0
        prof_lessons = {}

        for (course, lesson), t_idx in teachers.items():
            prof_name = int_to_teacher[t_idx]
            room_idx = rooms[(course, lesson)]
            room_name = int_to_room[room_idx]
            prof_lessons.setdefault(prof_name, []).append((slots[(course, lesson)], room_name))

        for lessons in prof_lessons.values():
            lessons_sorted = sorted(lessons, key=lambda x: x[0])
            last_room = None
            for _, room in lessons_sorted:
                if last_room is not None and room != last_room:
                    score += 1
                last_room = room

        if self.best_score is None or score < self.best_score:
            self.best_score = score
            self.best_solution = {
                "slots": slots,
                "rooms": rooms,
                "teachers": teachers
            }


# ======================================================
# 10. OUTPUT AND RESULTS
# ======================================================

print("\n--- STARTING SEARCH (OR-Tools CP-SAT) ---")
print(f"Heuristics: {heuristica_vars_nome}/{heuristica_vals_nome} / Timeout: {MAX_TIME_SECONDS}s\n")

collector = SolutionCollector(slot_vars, room_vars, teacher_vars)
start = time.time()
status = solver.SearchForAllSolutions(model, collector)
elapsed = time.time() - start

print("Execution time:", elapsed, "seconds")
print("Status:", solver.StatusName(status))
print("Total solutions found:", collector.solution_count)
print("Best solution score (fewer room changes):", collector.best_score)

output_filepath = r"outputs\best_schedule.json"

if collector.solution_count > 0:
    print(f"\nSolution found. Exporting best to {output_filepath}")
    schedule = {}
    best = collector.best_solution

    for (course, lesson), s_val in best["slots"].items():
        lesson_key = f"{course}_{lesson}"
        schedule.setdefault(lesson_key, {})["slot"] = int(s_val)

    for (course, lesson), r_val in best["rooms"].items():
        lesson_key = f"{course}_{lesson}"
        room_idx = int(r_val)
        schedule.setdefault(lesson_key, {})["room"] = int_to_room.get(room_idx, f"room_{room_idx}")

    for (course, lesson), t_val in best["teachers"].items():
        lesson_key = f"{course}_{lesson}"
        teacher_idx = int(t_val)
        schedule.setdefault(lesson_key, {})["teacher"] = int_to_teacher.get(teacher_idx, f"teacher_{teacher_idx}")

    sorted_schedule = dict(sorted(schedule.items(), key=lambda x: x[0]))
    with open(output_filepath, "w", encoding="utf-8") as f:
        json.dump(sorted_schedule, f, indent=4)

    print("Successfully exported to JSON (ordered by UC).")
    print("\n--- Example of Assignments (first 30) ---")
    all_vars_print = []
    for (course, lesson), s_val in best["slots"].items():
        all_vars_print.append((f"{course}_{lesson}_slot", s_val))
    for (course, lesson), r_val in best["rooms"].items():
        all_vars_print.append((f"{course}_{lesson}_room", int_to_room.get(r_val)))
    for (course, lesson), t_val in best["teachers"].items():
        all_vars_print.append((f"{course}_{lesson}_teacher", int_to_teacher.get(t_val)))
    all_vars_print = sorted(all_vars_print, key=lambda x: x[0])
    for i, (name, val) in enumerate(all_vars_print):
        if i < 30:
            print(f"{name} → {val}")
        else:
            print("...")
else:
    print("No solution found.")



--- STARTING SEARCH (OR-Tools CP-SAT) ---
Heuristics: mrv/lcv / Timeout: 30.0s

Execution time: 30.004518270492554 seconds
Status: FEASIBLE
Total solutions found: 19954
Best solution score (fewer room changes): 20

Solution found. Exporting best to outputs\best_schedule.json
Successfully exported to JSON (ordered by UC).

--- Example of Assignments (first 30) ---
UC11_1_room → Room2
UC11_1_slot → 14
UC11_1_teacher → jo
UC11_2_room → Room2
UC11_2_slot → 7
UC11_2_teacher → jo
UC12_1_room → Room1
UC12_1_slot → 5
UC12_1_teacher → mike
UC12_2_room → Room1
UC12_2_slot → 8
UC12_2_teacher → mike
UC13_1_room → Online
UC13_1_slot → 10
UC13_1_teacher → rob
UC13_2_room → Online
UC13_2_slot → 12
UC13_2_teacher → rob
UC14_1_room → Lab01
UC14_1_slot → 19
UC14_1_teacher → rob
UC14_2_room → Lab01
UC14_2_slot → 13
UC14_2_teacher → rob
UC15_1_room → Room1
UC15_1_slot → 2
UC15_1_teacher → sue
UC15_2_room → Online
UC15_2_slot → 15
UC15_2_teacher → sue
...
...
...
...
...
...
...
...
...
...
...
...
...
..